In [1]:
import csv
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load the csv file
print('loading the data...')
data=pd.read_csv('Behavioral_Cloning_Project/data/driving_log.csv')
print(data)
lines=np.asarray(data)

loading the data...
                                      center  \
0     IMG/center_2016_12_01_13_30_48_287.jpg   
1     IMG/center_2016_12_01_13_30_48_404.jpg   
2     IMG/center_2016_12_01_13_31_12_937.jpg   
3     IMG/center_2016_12_01_13_31_13_037.jpg   
4     IMG/center_2016_12_01_13_31_13_177.jpg   
...                                      ...   
8031  IMG/center_2016_12_01_13_46_38_745.jpg   
8032  IMG/center_2016_12_01_13_46_38_802.jpg   
8033  IMG/center_2016_12_01_13_46_38_846.jpg   
8034  IMG/center_2016_12_01_13_46_38_922.jpg   
8035  IMG/center_2016_12_01_13_46_38_947.jpg   

                                       left  \
0      IMG/left_2016_12_01_13_30_48_287.jpg   
1      IMG/left_2016_12_01_13_30_48_404.jpg   
2      IMG/left_2016_12_01_13_31_12_937.jpg   
3      IMG/left_2016_12_01_13_31_13_037.jpg   
4      IMG/left_2016_12_01_13_31_13_177.jpg   
...                                     ...   
8031   IMG/left_2016_12_01_13_46_38_745.jpg   
8032   IMG/left_2016_12_01_

In [3]:
images = []
measurements = []
for line in lines :
  source_path = line[0] 
  file_name = source_path.split('/')[-1]
  current_path = './Behavioral_Cloning_Project/data/IMG/' + file_name
  image = cv2.imread(current_path)
  images.append(image)
  measurement = float(line[3])
  measurements.append(measurement)

In [4]:
X_train = np.array(images)
y_train = np.array(measurements)

In [5]:
X_train

array([[[[208, 163, 130],
         [208, 163, 130],
         [208, 163, 130],
         ...,
         [210, 163, 131],
         [210, 163, 131],
         [210, 163, 131]],

        [[208, 163, 130],
         [208, 163, 130],
         [208, 163, 130],
         ...,
         [210, 163, 131],
         [210, 163, 131],
         [210, 163, 131]],

        [[209, 164, 131],
         [209, 164, 131],
         [209, 164, 131],
         ...,
         [211, 164, 132],
         [211, 164, 132],
         [211, 164, 132]],

        ...,

        [[ 83,  99, 112],
         [ 88, 104, 117],
         [ 89, 106, 119],
         ...,
         [ 99, 116, 135],
         [106, 123, 142],
         [111, 128, 147]],

        [[ 92, 109, 122],
         [ 88, 105, 118],
         [ 82,  99, 112],
         ...,
         [104, 123, 144],
         [ 96, 115, 136],
         [ 91, 110, 131]],

        [[ 87, 104, 117],
         [ 79,  96, 109],
         [ 79,  95, 111],
         ...,
         [115, 134, 155],
        

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout , Conv2D
from keras.layers import Lambda, Cropping2D

In [6]:
CROP_TOP = 70
CROP_BOTTOM = 25

model = Sequential()

'''
Lambda Layer allows to specify the operation  ( normalization ) 
to be applied as a function that operate on each image as it passes through the layer

'''
# Preprocess incoming data ( normalization )
model.add(Lambda(lambda x: (x / 255.0) - 0.5,input_shape=(160,320,3)))

'''
the Cropping Layer crop the input images by 
    - 70 rows pixels from the top image
    - 25 rows pixels from the bottom image
    - 0 Columns pixels from the left of the  image
    - 0 Columns pixels from the right of the image

'''
# crops at top and bottom, ROI ( output ) shape = (75, 320, 3)
model.add(Cropping2D(cropping=((CROP_TOP,CROP_BOTTOM), (0,0))))

# layer 1- Convolution layer with 24 filters , a 5x5 kernel, stride= 2x2
model.add(Conv2D( filters = 24 , kernel_size = (5,5) , strides=(2,2) ))
# Relu Activation Function 
model.add(Activation('relu'))

# layer 2- Convolution layer with 36 filters , a 5x5 kernel, stride= 2x2
model.add(Conv2D( filters = 36 , kernel_size = (5,5) , strides=(2,2) ))
model.add(Activation('relu'))

# layer 3- Convolution layer with 48 filters , a 5x5 kernel, stride= 2x2
model.add(Conv2D( filters = 48 , kernel_size = (5,5) , strides=(2,2) ))
model.add(Activation('relu'))

# layer 4- Convolution layer with 64 filters , a 3x3 kernel, stride= 1x1
model.add(Conv2D( filters = 64 , kernel_size = (3,3) ))
model.add(Activation('relu'))

# layer 5- Convolution layer with 64 filters , a 3x3 kernel, stride= 1x1
model.add(Conv2D( filters = 64 , kernel_size = (3,3) ))
model.add(Activation('relu'))

# flatten image from 2D to side by side
model.add(Flatten())

# layer 6- fully connected layer 1
model.add(Dense(100))
model.add(Activation('relu'))

# Adding a dropout layer to avoid overfitting. Here we are have given the dropout rate as 25% after first fully connected layer
model.add(Dropout(0.25))

# layer 7- fully connected layer 2
model.add(Dense(50))
model.add(Activation('relu'))


# layer 8- fully connected layer 3
model.add(Dense(10))
model.add(Activation('relu'))

# layer 9- fully connected layer 4
model.add(Dense(1)) 


# the output is the steering angle
# using mean squared error loss function 
model.compile(optimizer='adam', loss='mse')

#fit generator is used here as the number of images are generated by the generator
# no of epochs : 5

model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator,   nb_val_samples=len(validation_samples), nb_epoch=5, verbose=1)

#saving model
model.save('model.h5')

print('Done! Model Saved!')

# keras method to print the model summary
model.summary()




model.fit(X_train , y_train , validation_split = 0.2, shuffle =True , nb_epoch = 7 )

model.save('model.h5')


Using TensorFlow backend.
d:\PyyMol\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\PyyMol\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\PyyMol\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\PyyMol\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synon


Train on 6428 samples, validate on 1608 samples
Epoch 1/7
6428/6428 [==============================] - 3s 445us/step - loss: 8061698.8113 - val_loss: 3894.7934
Epoch 2/7
6428/6428 [==============================] - 3s 433us/step - loss: 4071.3980 - val_loss: 2206.4868
Epoch 3/7
6428/6428 [==============================] - 3s 430us/step - loss: 2612.7248 - val_loss: 2239.4255
Epoch 4/7
6428/6428 [==============================] - 3s 440us/step - loss: 2310.4237 - val_loss: 2957.4433
Epoch 5/7
6428/6428 [==============================] - 3s 438us/step - loss: 2268.1963 - val_loss: 1577.0872
Epoch 6/7
6428/6428 [==============================] - 3s 437us/step - loss: 1882.8759 - val_loss: 1960.4701
Epoch 7/7
6428/6428 [==============================] - 3s 441us/step - loss: 2171.5728 - val_loss: 3680.2609
